In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

Mounted at /content/drive


In [2]:
from loader_util.preprocessing import ImageToArrayPreprocessor, \
    SimplePreProcessor, MeanPreprocessor, CropPreprocessor
from loader_util.io import HDF5DatasetGenerator
from loader_util.utils.ranked import rank5_accuracy
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
import numpy as np
import progressbar
import json

In [3]:
json_path = r"/content/drive/MyDrive/ImageDatasets/cats_dogs_hdf5/output/alexnet_dogs_vs_cats_mean.json"
model_path = r"/content/drive/MyDrive/ImageDatasets/cats_dogs_hdf5/model/saved_model/alexnet_dogs_cats.model"
test_path = r"/content/drive/MyDrive/ImageDatasets/cats_dogs_hdf5/test.hdf5"

# load the RGB means for the train set
means = json.loads(open(json_path).read())

# init the image preprocessors
sp = SimplePreProcessor(227, 227)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
cp = CropPreprocessor(227, 227)
iap = ImageToArrayPreprocessor()

# load the pretrained network
print(f"[INFO] loading model......")
model = load_model(model_path) # type: Model

[INFO] loading model......


In [4]:
# init the testing dataset generator then make prediction on the testing data
print(f"[INFO] prediction on test data (no crops)......")
testgen = HDF5DatasetGenerator(dbPath=test_path,
                               batchSize=64,
                               classes=2,
                               preprocessors=[sp, mp, iap])

# get the predictions
preds = model.predict_generator(testgen.generator(),
                                steps=testgen.numImages // 64,
                                max_queue_size=10)


# computer the rank5 accuracies
rank1, _ = rank5_accuracy(preds, testgen.db["labels"])
print(f"[INFO] rank1 accuracy: {rank1: 0.2f}......")
testgen.close()

[INFO] prediction on test data (no crops)......


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


[INFO] rank1 accuracy:  0.93......


In [5]:
# reinit the test set generator for crop preprocessing
testgen = HDF5DatasetGenerator(test_path,
                               batchSize=64,
                               preprocessors=[mp],
                               classes=2,)
predictions = []


# init the progressbar
# initialize the progress bar
widgets = ["Building Dataset: ", progressbar.Percentage(), " ",
           progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=testgen.numImages // 64,
                               widgets=widgets).start()

# loop over the single pass of the test data
for i, (images, labels) in enumerate(testgen.generator(passes=1)):
    # we just got back a batch of image from the test data
    for image in images:
        crops = cp.preprocess(image)
        crops = np.array([iap.preprocess(c) for c in crops], dtype="float32")

        # make predictions on the crops and then average them together
        pred = model.predict(crops)
        predictions.append(pred.mean(axis=0))

    pbar.update(i) # gets updated every batch
pbar.finish()

Building Dataset: 100% |#######################################| Time:  0:02:31


In [6]:
print(f"[INFO] predicting on test data with crops......")
rank1, _ = rank5_accuracy(predictions, testgen.db["labels"])
print(f"[INFO] rank1 accuracy = {rank1: 0.2f}......")
testgen.close()

[INFO] predicting on test data with crops......
[INFO] rank1 accuracy =  0.94......
